# Imports and bronze table reading

In [0]:
import pyspark.sql.functions as func
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")
#df.display()

# Transformations

### Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

### Normalization

In [0]:
df = (
    df
    .withColumn(
        "cst_marital_status",
        func.when(func.upper(func.col("cst_marital_status")) == "S", "Single")
         .when(func.upper(func.col("cst_marital_status")) == "M", "Married")
         .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        func.when(func.upper(func.col("cst_gndr")) == "F", "Female")
         .when(func.upper(func.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )
)

### Removing Records with Missing Customer ID

In [0]:
df = df.filter(col("cst_id").isNotNull())

### Renaming Columns

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

### Sanity checks

In [0]:
df.limit(10).display()

# Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_customers")

### Sanity Check of Silver Table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_customers LIMIT 10